# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('../output_data/city_data.csv')
city_data.head()

,Unnamed: 0,Origin,Lat,Long,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date
0,0,avarua,-21.21,-159.78,82.40,69,79,10.29,1587266413
1,1,luderitz,-26.65,15.16,59.00,93,10,8.05,1587266413
2,2,druzhba,52.04,33.95,34.12,82,100,5.91,1587266414
3,3,lavrentiya,65.58,-171.00,20.55,96,100,9.66,1587266414
4,4,mogadishu,2.04,45.34,82.90,77,100,10.18,1587266414


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat", "Long"]]
humidity = city_data["Humidity %"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_low_temp = city_data.loc[city_data["Max Temp (F)"] > 50]
perfect_high_temp = perfect_low_temp.loc[perfect_low_temp ["Max Temp (F)"] < 70]
perfect_wind_speed = perfect_high_temp.loc[perfect_high_temp ["Wind Speed (mph)"] < 2]
perfect_humidity = perfect_wind_speed.loc[perfect_wind_speed["Humidity %"] < 80]
perfect_humidity

,Unnamed: 0,Origin,Lat,Long,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date
119,119,jurm,36.86,70.83,52.16,29,24,0.56,1587266430
153,153,gualeguaychu,-33.01,-58.52,63.00,77,0,1.99,1587266435
201,201,kyenjojo,0.63,30.62,62.71,66,100,1.12,1587266442
281,281,san cristobal,7.77,-72.22,68.52,63,77,1.34,1587266367
298,298,mondlo,-27.98,30.72,55.29,69,34,0.43,1587266459
304,304,aksu,41.12,80.26,60.78,29,100,1.07,1587266460
333,333,yura,-16.25,-71.68,56.10,26,34,0.63,1587266465
335,335,tekeli,44.83,78.82,52.20,25,0,1.52,1587266465
349,349,springbok,-29.66,17.89,54.01,29,98,1.83,1587266467
380,380,aiquile,-18.20,-65.18,50.45,39,0,1.25,1587266473


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

hotel_name = {'Unnamed: 0':[], 'Hotel Name':[]}
hotel_name_df = pd.DataFrame(data=hotel_name)
hotel_df = pd.merge(perfect_humidity, hotel_name_df, on="Unnamed: 0", how="outer")
hotel_df["Coordinates"] = hotel_df["Lat"].map(str) + ", " + hotel_df["Long"].map(str)
hotel_df


,Unnamed: 0,Origin,Lat,Long,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date,Coordinates,Hotel Name
0,119,jurm,36.86,70.83,52.16,29,24,0.56,1587266430,"36.86, 70.83",NaN
1,153,gualeguaychu,-33.01,-58.52,63.00,77,0,1.99,1587266435,"-33.01, -58.52",NaN
2,201,kyenjojo,0.63,30.62,62.71,66,100,1.12,1587266442,"0.63, 30.62",NaN
3,281,san cristobal,7.77,-72.22,68.52,63,77,1.34,1587266367,"7.77, -72.22",NaN
4,298,mondlo,-27.98,30.72,55.29,69,34,0.43,1587266459,"-27.98, 30.72",NaN
5,304,aksu,41.12,80.26,60.78,29,100,1.07,1587266460,"41.12, 80.26",NaN
6,333,yura,-16.25,-71.68,56.10,26,34,0.63,1587266465,"-16.25, -71.68",NaN
7,335,tekeli,44.83,78.82,52.20,25,0,1.52,1587266465,"44.83, 78.82",NaN
8,349,springbok,-29.66,17.89,54.01,29,98,1.83,1587266467,"-29.66, 17.89",NaN
9,380,aiquile,-18.20,-65.18,50.45,39,0,1.25,1587266473,"-18.2, -65.18",NaN


In [10]:

location = hotel_df["Coordinates"]
hotel_result = []
target_type = "hotel"
search_area = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": target_type,
    "radius" : search_area,
    "key": g_key}
for index, row in hotel_df.iterrows():
    hotel_coordinates = row["Coordinates"]
    params["keyword"] = target_type
    response = requests.get(base_url, params = params).json()
    results = response["results"]
    nearest_hotel = requests.get(base_url, params = params).json()
    hotel_result.append(nearest_hotel["results"][0]["name"])
    

IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
